In [1]:
import numpy as np
import os
from tqdm.auto import tqdm
# from pathos.multiprocessing import ProcessingPool as Pool
# from functools import partial
import pandas as pd
from PIL import Image

In [2]:
# clone Scaled_YOLOv4
!git clone https://github.com/tom-auger/ScaledYOLOv4.git
%cd ScaledYOLOv4/
#checkout the yolov4-large branch
!git checkout yolov4-large

Cloning into 'ScaledYOLOv4'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 220 (delta 61), reused 60 (delta 30), pack-reused 128
Receiving objects: 100% (220/220), 401.93 KiB | 1.58 MiB/s, done.
Resolving deltas: 100% (113/113), done.
/kaggle/working/ScaledYOLOv4
Branch 'yolov4-large' set up to track remote branch 'yolov4-large' from 'origin'.
Switched to a new branch 'yolov4-large'


In [3]:
df_train = pd.read_csv('/kaggle/input/vinbigstratifiedyolo/vinbigstratified/train_orig.csv')
df_val = pd.read_csv('/kaggle/input/vinbigstratifiedyolo/vinbigstratified/validation_orig.csv')
df_test = pd.read_csv('/kaggle/input/vinbigstratifiedyolo/vinbigstratified/test_orig.csv')

In [4]:
import torch
print('Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Using torch 1.7.0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


In [5]:
#install mish activation funciton for cuda
%cd ..
!git clone https://github.com/JunnYu/mish-cuda
%cd mish-cuda
!python setup.py build install
%cd ..

/kaggle/working
Cloning into 'mish-cuda'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 195 (delta 7), reused 79 (delta 3), pack-reused 107
Receiving objects: 100% (195/195), 208.77 KiB | 1.22 MiB/s, done.
Resolving deltas: 100% (56/56), done.
/kaggle/working/mish-cuda
/opt/conda/lib/python3.7/distutils/extension.py:131: UserWarning: Unknown Extension options: 'headers'
  warnings.warn(msg)
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.7
creating build/lib.linux-x86_64-3.7/mish_cuda
copying src/mish_cuda/__init__.py -> build/lib.linux-x86_64-3.7/mish_cuda
running egg_info
creating src/mish_cuda.egg-info
writing src/mish_cuda.egg-info/PKG-INFO
writing dependency_links to src/mish_cuda.egg-info/dependency_links.txt
writing requirements to src/mish_cuda.egg-info/requires.txt
writing top-level names to src/mish_cuda.egg-info/top_level.txt
writing manifes

In [6]:
%mkdir data
def write_to_file(imgs, dataset_name):
    with open(f"data/{dataset_name}_abnormsonly.txt", 'wt') as f:
        for img in imgs:
            f.write(f"/kaggle/input/vinbigstratifiedyolo/vinbigstratified/{dataset_name}/{img}.png\n")

train_imgs = df_train[df_train['class_id'] != 14]['image_id'].unique()
val_imgs = df_val[df_val['class_id'] != 14]['image_id'].unique()
test_imgs = df_test[df_test['class_id'] != 14]['image_id'].unique()

write_to_file(train_imgs, 'train')
write_to_file(val_imgs, 'validation')
write_to_file(test_imgs, 'test')

In [7]:
with open(f"data.yaml", 'wt') as f:
        f.write(f"""train: ../data/train_abnormsonly.txt
val: ../data/validation_abnormsonly.txt

nc: 14
names: ['Aortic enlargement', 'Atelectasis', 'Calcification', 'Cardiomegaly', 'Consolidation', 'ILD', 'Infiltration', 'Lung Opacity', 'Nodule/Mass', 'Other lesion', 'Pleural effusion', 'Pleural thickening', 'Pneumothorax', 'Pulmonary fibrosis']
""")

In [8]:
with open(f"data_all.yaml", 'wt') as f:
        f.write(f"""train: /kaggle/input/vinbigstratifiedyolo/vinbigstratified/train
val: /kaggle/input/vinbigstratifiedyolo/vinbigstratified/validation

nc: 14
names: ['Aortic enlargement', 'Atelectasis', 'Calcification', 'Cardiomegaly', 'Consolidation', 'ILD', 'Infiltration', 'Lung Opacity', 'Nodule/Mass', 'Other lesion', 'Pleural effusion', 'Pleural thickening', 'Pneumothorax', 'Pulmonary fibrosis']
""")

In [9]:
%cd ScaledYOLOv4/

/kaggle/working/ScaledYOLOv4


In [10]:
!python test.py --iou-thres 0.65 --conf 0.001 --verbose --labels 'labels_all' --img 640 --batch 16 --data '../data.yaml' --weights '/kaggle/input/mlpyoloweighted-labels-all-20-40/ScaledYOLOv4/runs/exp0_yolov4-csp-results/weights/best_yolov4-csp-results.pt'

Namespace(augment=False, batch_size=16, conf_thres=0.001, data='../data.yaml', device='', filterclass=-1, img_size=640, iou_thres=0.65, labels='labels_all', merge=False, onlyclass=-1, save_json=False, save_txt=False, single_cls=False, task='val', verbose=True, weights=['/kaggle/input/mlpyoloweighted-labels-all-20-40/ScaledYOLOv4/runs/exp0_yolov4-csp-results/weights/best_yolov4-csp-results.pt'])
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

Fusing layers... Model Summary: 235 layers, 5.25406e+07 parameters, 5.04494e+07 gradients
Scanning images: 100%|████████████████████████| 460/460 [00:09<00:00, 48.41it/s]
Scanning labels /kaggle/working/cache/labels_all (460 found, 0 missing, 0 empty,
               Class      Images     Targets           P           R      mAP@.5
                 all         460     3.7e+03       0.358       0.358        0.32       0.134
  Aortic enlargement         460         737       0.685       0.533       0.669   

In [11]:
!python test.py --iou-thres 0.65 --conf 0.001 --verbose --labels 'labels_all' --img 640 --batch 16 --data '../data.yaml' --weights '/kaggle/input/mlpyoloabnormsonly-labels-all/ScaledYOLOv4/runs/exp0_yolov4-csp-results/weights/best_yolov4-csp-results.pt'

Namespace(augment=False, batch_size=16, conf_thres=0.001, data='../data.yaml', device='', filterclass=-1, img_size=640, iou_thres=0.65, labels='labels_all', merge=False, onlyclass=-1, save_json=False, save_txt=False, single_cls=False, task='val', verbose=True, weights=['/kaggle/input/mlpyoloabnormsonly-labels-all/ScaledYOLOv4/runs/exp0_yolov4-csp-results/weights/best_yolov4-csp-results.pt'])
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

Fusing layers... Model Summary: 235 layers, 5.25406e+07 parameters, 5.04494e+07 gradients
Scanning labels /kaggle/working/cache/labels_all (460 found, 0 missing, 0 empty,
               Class      Images     Targets           P           R      mAP@.5
                 all         460     3.7e+03       0.361       0.364        0.33       0.139
  Aortic enlargement         460         737       0.629       0.552       0.677       0.376
         Atelectasis         460          29       0.492        0.31      